# Calculate the Standard Deviation of Residuals
Standard deviation of the stage residual values is calculated from the feild measurements, rating curve, and valid gauges, and is corresoponding to the standard deviation of a gauges stage residual values calculated in other files.
#### Input:
- field_measurements.
- rating_curve.
- valid gages.

#### Output:
- std of all gages saved as a csv file.

Note that this code is used in the scope of this project, but can be adapted for other gages.

In [1]:
import pandas as pd
import numpy as np
field_measurements = pd.read_csv("field_measurements.csv", encoding='latin-1', low_memory=False)
field_measurements.reset_index(drop=True)
curve = pd.read_csv('rating_curve_wa.csv', encoding='latin-1', low_memory=False)
ca_gages = pd.read_csv("CA_valid_gages.csv", encoding = 'latin-1', low_memory=False)["Site No"]

In [2]:
std_df = pd.DataFrame(columns=['Site No', 'residual_std'])
for site_name in ca_gages:
    site_d = field_measurements.loc[field_measurements['site_no'] == site_name]
    curve_d = curve.loc[curve['Site No'] == site_name]
    if (not site_d.empty and not curve_d.empty):

        curve_d = curve_d.dropna(subset=["INDEP", "DEP"])

        stage_rating = curve_d["INDEP"].to_numpy()
        Q_rating = curve_d["DEP"].to_numpy()

        Q = site_d["discharge_va"].to_numpy()
        h_gage = site_d["gage_height_va"].to_numpy()

        stage_rating_interp = np.interp(Q, Q_rating, stage_rating)
        stage_residual = h_gage - stage_rating_interp

        residual_std = np.nanstd(stage_residual)
        std_df = std_df.append({'Site No': site_name, 'residual_std': residual_std}, ignore_index=True)

C:\Users\zicon\AppData\Local\Temp\ipykernel_15336\238925637.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  std_df = std_df.append({'Site No': site_name, 'residual_std': residual_std}, ignore_index=True)
C:\Users\zicon\AppData\Local\Temp\ipykernel_15336\238925637.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  std_df = std_df.append({'Site No': site_name, 'residual_std': residual_std}, ignore_index=True)
C:\Users\zicon\AppData\Local\Temp\ipykernel_15336\238925637.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  std_df = std_df.append({'Site No': site_name, 'residual_std': residual_std}, ignore_index=True)
C:\Users\zicon\AppData\Local\Temp\ipykernel_15336\238925637.py:19: FutureWarning: The frame.append method 

In [3]:
std_df

,Site No,residual_std
0,10260500.0,0.577824
1,10261500.0,3.859658
2,10263000.0,11.409305
3,10290500.0,0.168970
4,10291500.0,0.114744
...,...,...
95,11478500.0,0.961486
96,11481000.0,1.482826
97,11481200.0,0.500469
98,11482500.0,2.285525


In [69]:
std_df.to_csv('std.csv', index=False)